In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec


pc = Pinecone(api_key="pcsk_2bhDrj_71BTyJqtUyjLFsufHZfnDHeSuLVFx4yATrd1eLWnq6mQkJM9LGD6Cwg4R11hbrG")

if "cutomer" not in pc.list_indexes().names():
    pc.create_index(
        name="cutomer",
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-west-2")
    )


index = pc.Index("cutomer")



In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_2bhDrj_71BTyJqtUyjLFsufHZfnDHeSuLVFx4yATrd1eLWnq6mQkJM9LGD6Cwg4R11hbrG")
index = pc.Index("cutomer")

In [ ]:
import os
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer


pc = Pinecone(api_key="pcsk_2bhDrj_71BTyJqtUyjLFsufHZfnDHeSuLVFx4yATrd1eLWnq6mQkJM9LGD6Cwg4R11hbrG")
index_name = "customer"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",

    )


index = pc.Index(index_name)


model = SentenceTransformer('all-MiniLM-L6-v2')


def generate_embedding(text):
    return model.encode(text).tolist()


vectors = [
    {
        "id": "vec1",
        "values": generate_embedding("Thank you for contacting us regarding the connectivity issue you're experiencing with {product_name} ({issue_description}). To resolve this, please try the following troubleshooting steps: 1. Restart your modem/router. 2. Check the cable connections and ensure everything is secure. 3. Confirm that your service plan is active and there are no outstanding dues. If the issue persists after these steps, please let us know, and we will escalate this to our technical team for further investigation. Best regards, {company_name} Support Team"),
        "metadata": {"issue": "Connectivity issue troubleshooting steps"}
    },
    {
        "id": "vec2",
        "values": generate_embedding("Dear {customer_name}, Thank you for contacting us about the billing issue with {product_name}. We are sorry for the inconvenience caused. We have reviewed your account, and it seems the issue is related to {billing_issue}. Please allow us {processing_time} business days to investigate further. We will notify you once the issue has been resolved. If you have any urgent concerns, feel free to contact us. Best regards, {company_name} Billing Support"),
        "metadata": {"issue": "Billing issue response"}
    }
]


index.upsert(
    vectors=vectors,
    namespace="ns1"
)

print("Data upserted successfully!")

# Incoming issue text
incoming_issue = "Customer is experiencing issues with connectivity on the product."

# Generate the embedding for the incoming issue
incoming_embedding = generate_embedding(incoming_issue)

# Perform similarity search to find the most similar issues in the index
results = index.query(
    vector=incoming_embedding,
    top_k=3,
    include_metadata=True,
    namespace="ns1"
)

# Display results
for result in results['matches']:
    print(f"ID: {result['id']}, Similarity Score: {result['score']}, Issue: {result['metadata']['issue']}")


Data upserted successfully!
